<a href="https://colab.research.google.com/github/RAgassi5/IntroToAi/blob/main/AgenticAI_Planning_Prolem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agentic AI

## Prerequisites

### Install a Local LLM with Ollama

To run this project locally, we will install and use **Ollama**, a lightweight runtime for local large language models.

**Download Ollama:**  
https://ollama.com/

Once installed, you can pull any model you want to run.  
Below are a few recommended examples, but you are free to pick any size or model from the Ollama library.

ollama pull qwen3:0.6b

or

ollama pull ibm/granite4:350m

or

Choose any model you prefer, make sure the model supports tools.
Browse available models here:
https://ollama.com/library



### Python requirements

In [ ]:
!pip install langgraph langchain-google-genai langchain-core mcp langchain-ollama

In [ ]:
!pip install unified-planning[pyperplan]

In [ ]:
import os
print("Current working directory:", os.getcwd())
print("Files found:", os.listdir())
# You should see your .pddl files in this list

Current working directory: /Users/roiiagassi/Documents/BGU/Year 3.1/Intro to AI/AgenticAI 
Files found: ['assignment_4_project_agentic.pdf', 'AgenticAI_Planning_Prolem1.ipynb', '.DS_Store', 'community_garden.pddl', 'community_garden_problem_v1.pddl', 'AgenticAI_Planning_Problem5.ipynb', 'community_garden_problem_v4.pddl', 'community_garden_problem_v5.pddl', 'community_garden_problem_v2.pddl', 'IntroToAI3_CopyForAI4', 'uni_planner.py', 'community_garden_problem_v3.pddl', '.idea']


## 1. Define FastMCP Tools

In [ ]:
from mcp.server.fastmcp import FastMCP
import math
from unified_planning import Environment
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import OneshotPlanner

# Initialize FastMCP
mcp = FastMCP("Unified Solver")

@mcp.tool()
def solve_planning_problem() -> str:
    """
    Runs the deterministic PDDL planner code to solve the Community Garden problem.
    """
    try:
        env = Environment()
        reader = PDDLReader(env)

        # UPDATE THESE FILENAMES to match exactly what you uploaded
        problem = reader.parse_problem("./community_garden.pddl", "./community_garden_problem_v1.pddl")

        with OneshotPlanner(name="pyperplan") as planner:
            result = planner.solve(problem)

        if result.plan is not None:
            output = "Plan found:\n"
            for action_instance in result.plan.actions:
                output += str(action_instance) + "\n"
            return output
        else:
            return "No plan found."
    except Exception as e:
        return f"Error running planner: {str(e)}"

# TO DO: Add more tools as needed for your application





## 2. LLM + MCP

### 2.1. Global instance of our LLM

In [ ]:
import os
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI


# Choose your model here, can be Ollama or Google Gemini
# Can also switch between different model sizes as needed
#model = "qwen3:0.6b"
#model = "ibm/granite4:350m"
model = "qwen2.5:3b"
global_llm = ChatOllama(model=model, temperature=0.0)

# SETUP API KEY if using Google Gemini
#os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY_HERE"

# model = "gemini-2.5-flash"
# model = "gemini-2.5-flash-lite"
# global_llm = ChatGoogleGenerativeAI(model=model, temperature=0)


### 2.2. Our agent graph

In [ ]:
from langgraph.graph import MessagesState, START, StateGraph
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver # Optional: For saving graph state


def create_agent_graph(sys_msg, tools):
    """ Creates a LangGraph StateGraph with the given tools integrated."""

    llm = global_llm

    if tools:
        llm_with_tools = llm.bind_tools(tools)
    else:
        llm_with_tools = llm

    # Node
    def assistant(state: MessagesState):
        return {
            "messages": [
                llm_with_tools.invoke([sys_msg] + state["messages"], think=False)
            ]
        }

    # Graph
    builder = StateGraph(MessagesState)

    # Define the basic graph structure
    builder.add_node("assistant", assistant)
    builder.add_edge(START, "assistant")

    if tools:
        builder.add_node("tools", ToolNode(tools))
        builder.add_conditional_edges(
            "assistant",
            tools_condition,
        )
        builder.add_edge("tools", "assistant")

    react_graph = builder.compile()

    return react_graph


async def run_agent(prompt, tools, sys_msg=""):

    sys_msg = SystemMessage(content=sys_msg)

    # 3. Create Graph
    graph = create_agent_graph(sys_msg, tools)

    # 4. Run (using ainvoke for async tools)
    config = {"configurable": {"thread_id": "1"}}
    result = await graph.ainvoke({"messages": [HumanMessage(content=prompt)]}, config)

    last_msg = result["messages"][-1].content

    # Extract tool names and outputs
    tools_used = []
    tools_output = []

    # Parsing logic specific to your request
    for msg in result["messages"]:
        # In LangChain, tool calls are usually in 'tool_calls' attribute of AIMessage
        # or 'name' attribute if it is a ToolMessage
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
             for tool_call in msg.tool_calls:
                tools_used.append(tool_call['name'])

        if msg.type == 'tool':
            tools_output.append(msg.content)

    return last_msg, tools_used, tools_output

### 2.3. Tools that run spacific agent (with tools and without)

In [ ]:

@mcp.tool()
async def ask_agent_with_tools(prompt) -> str:
    """ Runs the agent that has access to the deterministic PDDL planner tool."""
    # This agent has access to the 'solve_planning_problem' tool defined above
    tools = [solve_planning_problem]
    results = await run_agent(prompt, tools)
    return results[0]

try:
    with open("community_garden.pddl", "r") as f:
        domain_text = f.read()
    with open("community_garden_problem_v1.pddl", "r") as f:
        problem_text = f.read()
except FileNotFoundError:
    print("Error: Please make sure the .pddl files are in the notebook directory.")
    domain_text = "Domain file not found."
    problem_text = "Problem file not found."

@mcp.tool()
async def ask_agent_without_tools(prompt) -> str:
    """ Runs the agent WITHOUT tools. It must try to solve the problem using only the PDDL description."""

    # We inject the text we just read into the prompt
    pddl_context = f"""
    You are a planning algorithm.
    Here is the PDDL Domain:
    {domain_text}

    Here is the PDDL Problem:
    {problem_text}

    Based ONLY on this text, generate a valid plan to reach the goal.
    List the actions step-by-step.
    """

    full_prompt = f"{pddl_context}\n\nUser Request: {prompt}"

    results = await run_agent(full_prompt, [])
    return results[0]

## 3. Run the Test

In [ ]:
# THE JUDGE AGENT RUNNER

sys_msg = """
    You are a Research Supervisor for an AI Planning course. You have two assistants:
    1. 'ask_agent_with_tools': runs a verified PDDL planner algorithm (Ground Truth).
    2. 'ask_agent_without_tools': is an LLM attempting to solve the problem intuitively.

    Your Goal:
    1. Call 'ask_agent_with_tools' to get the mathematically correct plan for the Community Garden problem.
    2. Call 'ask_agent_without_tools' to see how the LLM attempts to solve it.
    3. Compare the two plans. Did the LLM hallucinate impossible actions? Did it miss preconditions?
    4. Provide a final summary explaining the differences.
    """

prompt = "I need a through comparison report for the planning problem. Run both agents and compare their plans."

tool_list = [ask_agent_with_tools, ask_agent_without_tools]

response, tools, outputs = await run_agent(prompt, tool_list, sys_msg)
print(f"Response: {response}")
print(f"Tools Used: {tools}")
print(f"Tool Outputs: {outputs}")


[01/18/26 10:50:28] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=41736;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=395502;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\

[01/18/26 10:52:18] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=692754;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=674856;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 22 of `/var/folders/9t/qq0sfdr93rj8j4thhgljn77c0000gq/T/ipykernel_16466/470738424.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.



[01/18/26 10:52:49] INFO     Grounding start: instance_7                                             ]8;id=18220;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/planner.py\planner.py]8;;\:]8;id=30480;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/planner.py#128\128]8;;\

                    INFO     Relevance analysis removed 0 facts                                    ]8;id=690791;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/grounding.py\grounding.py]8;;\:]8;id=485922;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/grounding.py#160\160]8;;\

...

                    INFO     66 Nodes expanded                                                        ]8;id=192626;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/search/a_star.py\a_star.py]8;;\:]8;id=332227;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/search/a_star.py#157\157]8;;\

                    INFO     Search end: instance_7                                                  ]8;id=855786;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/planner.py\planner.py]8;;\:]8;id=407786;file:///opt/anaconda3/lib/python3.12/site-packages/pyperplan/planner.py#147\147]8;;\

[01/18/26 10:55:46] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=538581;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=634415;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\

[01/18/26 10:58:10] INFO     HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"   ]8;id=972577;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=777423;file:///opt/anaconda3/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\

Response: Based on the plans provided by both agents, here is a comparison:

### Agent with Tools (ask_agent_with_tools)
- **Sequence of Actions:**
  1. Move volunteer1 to cell33, then move them to gardenplot.
  2. Move volunteer2 to cell12, then move them to gardenplot.
  3. Move volunteer3 to cell13, then move them to cell35 and finally to gardenplot.
  4. Get a watering can for volunteer3 at cell35 and bring it to the gardenplot.
  5. Move volunteer3 from cell35 to gardenplot.
  6. Move volunteer1 to gardenplot in cell30.
  7. Get a tiller for volunteer1 in cell30, then move them there.
  8. Move volunteer1 from cell30 to gardenplot.
  9. Till the soil using the tiller at gardenplot.
  10. Move volunteer2 to gardenplot in cell38.
  11. Get seeds for volunteer2 in cell38 and bring them to gardenplot.
  12. Move volunteer2 from cell38 to gardenplot.
  13. Sow the seeds in gardenplot.
  14. Water the garden using the watering can at gardenplot.
  15. Celebrate the opening of the garden